In this notebook, we will run and verify the SWOT Assimilated DiScharge (SADS) algorithm against data from the SWOT Pepsi-1 challenge. We start by adding the SADS modules to the load path and importing them. 

In [5]:
@everywhere push!(LOAD_PATH,"../src")
using Gvf
using Kalman
using NCDatasets
using Distributions
using Plots
plotly();  # set the plotting backend

We load the data from the Po dataset file

In [165]:
ds = Dataset("Po.nc");
xs = NCDatasets.group(ds, "XS_Timeseries");
ri = NCDatasets.group(ds, "River_Info");

In [166]:
qwbm = ri["QWBM"][1];
x = xs["X"][:]; x = x[end] - x; x = x[end:-1:1];
W = xs["W"][:]; W = W[end:-1:1, :];
H = xs["H"][:]; H = H[end:-1:1, :];
Q = xs["Q"][:]; Q = Q[end:-1:1, :];

The SADS algorithm consists of four components:
- Estimating effective bathymetry
- Preconditioning of ensemble discharge
- Estimation of effective roughness coefficient
- Estimation of discharge

We start by setting the ensemble size

In [167]:
nens = 100;

We then generate an ensemble of discharge, bed slopes, roughness coefficients. The slope is sampled from a uniform distribution bound by the observed water surface slope

In [168]:
Qe = rand(Normal(qwbm, 0.5*qwbm), nens);
ne = rand(Uniform(0.02, 0.05), nens);
S = diff(H, 1) ./ diff(x);
Se = rand(Uniform(minimum(S), maximum(S)), length(x), nens);

Then we can generate an ensemble of depth profiles using the GVF model after we calculate a downstream depth as the boundary condition. We use an assumption of normal flow to estimate that depth along with the observed water surface slope, which is
\begin{equation*}
y = \left( \dfrac{n^2 Q^2}{W^2 S_{f}} \right)^{3/10}
\end{equation*}
for a rectangular cross section

In [169]:
sf = mean((H[2,:]-H[1,:])./(x[2]-x[1]));
ye = (ne.^2 .* Qe.^2 ./ (mean(W[1, :])^2 .* sf)).^(3/10);

From the downstream depth and the ensemble slopes we can generate the thalweg profiles for each ensemble member

In [170]:
ze = zeros(length(x), nens);
ze[1, :] = minimum(H[1, :]) .- ye
for i in 2:68; ze[i, :] = ze[i-1, :] .+ Se[i, :] .* (x[i] - x[i-1]); end

The GVF model is then used to derive the steady-state water depth profiles

In [171]:
he = zeros(length(x), nens);
for i in 1:nens
    he[:, i] = gvf(Qe[i], ye[i], Se[:, i], ne[i], x, mean(W, 2));
end

We then assimilate the available water surface elevation (WSE) observations for different times until the bathymetry estimates converge

In [172]:
i = find(he[1, :] .> 0);
X = zeros(length(x), length(i));
X[:] = ze[:, i];
XA = zeros(length(x), length(i));
XA[:] = he[:, i] .+ ze[:, i];
zest = zeros(length(x), size(H, 2));
E = rand(Normal(0, 0.01), length(x), length(i));
for t in 1:size(H ,2)
    zest[:,t] = sqrtenkf(X, convert(Array{Float64, 1}, H[:, t]), XA, E);
end
plot(zest[end:-1:1, :], color="blue", legend=:none, ylabel="Bed Elevation (m)")

The bathymetry can be calculated as the mean of the estimated data, and the bed slope can be derived

In [16]:
z = mean(zest, 2);
S0 = diff(z) ./ diff(x);
S0 = [S0[1]; S0];

We can then precondition the discharge ensemble by using the LETKF and assimilating WSE observations for each time step

In [39]:
Qest = zeros(1, size(H, 2));
for t in 1:365
    sf = (H[2,t] - H[1,t]) / (x[2] - x[1]);
    ye = (ne.^2 .* Qe.^2 ./ (W[1,t]^2 .* sf)).^(3/10);
    for i in 1:100
       he[:,i] = gvf(Qe[i], ye[i], S0, ne[i], x, W[:,t]);
    end
    i = find(he[1,:].>0);
    E = rand(Normal(0, 1e-1), length(x), length(i));
    X = zeros(1, length(i));
    X[:] = Qe[i];
    XA = zeros(length(x), length(i));
    XA[:] = he[:, i] .+ z;
    d = zeros(length(x));
    d[:] = H[:, t];
    mapobs(i) = max(1, i-5):min(length(d), i+5);
    Qest[t] = letkf(X, d, XA, E , mapobs)[1,1];
end

Let's compare these initial estimates to the true discharge

In [40]:
Qobs = mean(Q, 1);
plot(Qobs', label="Truth")
plot!(Qest', label="SADS")
ylabel!("Discharge (m3/s)")
xlabel!("Day")

The next step includes the estimation of the effective roughness coefficient for each time step

In [64]:
n = zeros(size(H, 2));
for t in 1:size(H, 2)
he = zeros(length(x), nens);
ne = rand(Uniform(0.01, 0.08), nens);
for i in 1:nens; he[:, i] = gvf(Qest[t], H[1, t]-z[1], S0, ne[i], x, W[:, t]) .+ z; end
i = find(he[1, :] .> 0);
X = zeros(1, length(i));
X[:] = ne[i];
XA = zeros(length(x)-1, length(i));
XA[:] = he[2:end, i];
d = zeros(length(x)-1);
d[:] = H[2:end, t];
E = rand(Normal(0, 1e-2), length(d), length(i));
mapobs(i) = i;
n[t] = letkf(X, d, XA, E, mapobs)[1,1];
end

Let's compare the WSE profiles we get with the prior and posterior roughness coefficients 

In [68]:
t = 148;
hprior = gvf(Qest[t], H[1, t] - z[1], S0, mean(ne), x, W[:, t]) .+ z;
hposterior = gvf(Qest[t], H[1, t] - z[1], S0, n[t], x, W[:, t]) .+ z;
plot(hprior[end:-1:1], label="Prior")
plot!(hposterior[end:-1:1], label="Posterior")
plot!(H[end:-1:1, t], label="Observed")
ylabel!("Elevation (m)")

We can then perform the last step of the SADS algorithm and estimate the final discharge time series

In [136]:
Qsads = zeros(size(H, 2));
for t in 1:365
Qe = abs.(rand(Normal(Qest[t], 0.1*Qest[t]), nens));
he = zeros(length(x), nens);
for i in 1:nens; he[:, i] = gvf(Qe[i], H[1, t]-z[1], S0, n[t], x, W[:, t]) .+ z;end
i = find(he[1,:].>0);
X = zeros(1, length(i));
X[:] = Qe[i];
XA = zeros(length(x)-1, length(i));
XA[:] = he[2:end, i];
d = zeros(length(x)-1);
d[:] = H[2:end, t];
E = rand(Normal(0, 1e-1), length(x)-1, length(i));
mapobs(i) = max(1, i-5):min(length(d), i+5);
Qsads[t] = letkf(X, d, XA, E, mapobs)[1, 1];
end

In [173]:
plot(Qobs[1:365], label="Truth")
plot!(Qest[1:365], label="SADS-Initial")
plot!(Qsads[1:365], label="SADS")
ylabel!("Discharge (m3/s)")

Although the final step appears to improve the discharge estimates, there are some cases (especially at higher flows) when the initial estimates are better. Apart from exploring why this happens, some additional things to do include:
- Test SADS on rest of Pepsi-1 case studies
- Use trapezoidal cross sections, allowing varying widths
- Run SADS algorithm on Pepsi-2 datasets